In [6]:
!pip install keras

     |████████████████████████████████| 317kB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 25.1MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 276kB 1.7MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built pyyaml
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
#@title Download the pretrained keras model files
!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_wiki_tox_v3_model.h5 -o ./data/cnn_wiki_tox_v3_model.h5
!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_wiki_tox_v3_hparams.h5 -o ./data/cnn_wiki_tox_v3_hparams.h5
!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_wiki_tox_v3_tokenizer.pkl -o ./data/cnn_wiki_tox_v3_tokenizer.pkl

!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_debias_tox_v3_model.h5 -o ./data/cnn_debias_tox_v3_model.h5
!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_debias_tox_v3_hparams.h5 -o ./data/cnn_debias_tox_v3_hparams.h5
!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/cnn_debias_tox_v3_tokenizer.pkl -o ./data/cnn_debias_tox_v3_tokenizer.pkl

!curl -L https://storage.googleapis.com/what-if-tool-resources/computefest2019/wiki_test.csv -o ./data/wiki_test.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 59.5M  100 59.5M    0     0  3728k      0  0:00:16  0:00:16 --:--:-- 4113k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   227  100   227    0     0    373      0 --:--:-- --:--:-- --:--:--   373
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.7M  100 10.7M    0     0  2698k      0  0:00:04  0:00:04 --:--:-- 2698k      0  0:00:05  0:00:02  0:00:03 2006k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.6M  100 61.6M    0     0  3687k      0  0:00:17  0:00:17 --:--:-- 3988k   0     0  3

In [7]:
#@title Load the keras models
import sys
from keras.models import load_model
from six.moves import cPickle as pkl

def pkl_load(f):
  return pkl.load(f) if sys.version_info < (3, 0) else pkl.load(
    f, encoding='latin1')

model1 = load_model('data/cnn_wiki_tox_v3_model.h5')
with open('data/cnn_wiki_tox_v3_tokenizer.pkl', 'rb') as f:
  tokenizer1 = pkl_load(f)
tokenizer1.oov_token = None # quick fix for version issues

model2 = load_model('data/cnn_debias_tox_v3_model.h5')
with open('data/cnn_debias_tox_v3_tokenizer.pkl', 'rb') as f:
  tokenizer2 = pkl_load(f)
tokenizer2.oov_token = None # quick fix for version issues

Using TensorFlow backend.


In [8]:
#@title Define custom prediction functions so that WIT infers using keras models
from keras.preprocessing.sequence import pad_sequences

# Set up model helper functions:
PADDING_LEN = 250

# Get raw string out of tf.Example and prepare it for keras model input
def examples_to_model_in(examples, tokenizer):
  texts = [ex.features.feature['comment'].bytes_list.value[0] for ex in examples]
  if sys.version_info >= (3, 0):
    texts = [t.decode('utf-8') for t in texts]
  # Tokenize string into fixed length sequence of integer based on tokenizer 
  # and model padding
  text_sequences = tokenizer.texts_to_sequences(texts)
  model_ins = pad_sequences(text_sequences, maxlen=PADDING_LEN)
  return model_ins

# WIT predict functions:
def custom_predict_1(examples_to_infer):
  model_ins = examples_to_model_in(examples_to_infer, tokenizer1)
  preds = model1.predict(model_ins)
  return preds

def custom_predict_2(examples_to_infer):
  model_ins = examples_to_model_in(examples_to_infer, tokenizer2)
  preds = model2.predict(model_ins)
  return preds

In [9]:
#@title Define helper functions for dataset conversion from csv to tf.Examples
import numpy as np
import tensorflow as tf

# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None):
  examples = []
  if columns == None:
    columns = df.columns.values.tolist()
  for index, row in df.iterrows():
    example = tf.train.Example()
    for col in columns:
      if df[col].dtype is np.dtype(np.int64):
        example.features.feature[col].int64_list.value.append(int(row[col]))
      elif df[col].dtype is np.dtype(np.float64):
        example.features.feature[col].float_list.value.append(row[col])
      elif row[col] == row[col]:
        example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
    examples.append(example)
  return examples

# Converts a dataframe column into a column of 0's and 1's based on the provided test.
# Used to force label columns to be numeric for binary classification using a TF estimator.
def make_label_column_numeric(df, label_column, test):
  df[label_column] = np.where(test(df[label_column]), 1, 0)

In [11]:
#@title Read the dataset from CSV and process it for model {display-mode: "form"}
import pandas as pd

# Set the path to the CSV containing the dataset to train on.
csv_path = 'data/wiki_test.csv'

# Set the column names for the columns in the CSV. If the CSV's first line is a header line containing
# the column names, then set this to None.
csv_columns = None

# Read the dataset from the provided CSV and print out information about it.
df = pd.read_csv(csv_path, names=csv_columns, skipinitialspace=True)
df = df[['is_toxic', 'comment']]

# Remove non ascii characters
comments = df['comment'].values
proc_comments = []
for c in comments:
  try:
    if sys.version_info >= (3, 0):
      c = bytes(c, 'utf-8')
    c = c.decode('unicode_escape')
    if sys.version_info < (3, 0):
      c = c.encode('ascii', 'ignore')
    proc_comments.append(c.strip())
  except:
    proc_comments.append('')

df = df.assign(comment=proc_comments)

label_column = 'is_toxic'
make_label_column_numeric(df, label_column, lambda val: val)

examples = df_to_examples(df)

In [13]:
#@title Invoke What-If Tool for the data and two models (Note that this step may take a while due to prediction speed of the toxicity model){display-mode: "form"}
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder
num_datapoints = 1000  #@param {type: "number"}
tool_height_in_px = 720  #@param {type: "number"}

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(examples[:num_datapoints]).set_custom_predict_fn(
  custom_predict_1).set_compare_custom_predict_fn(custom_predict_2)

wv = WitWidget(config_builder, height=tool_height_in_px)
wv

WitWidget(config={'label_vocab': [], 'model_name_2': '2', 'model_type': 'classification', 'inference_address':…

#### Exploration ideas

- Organize datapoints by setting X-axis scatter to "inference score 1" and Y-axis scatter to "inference score 2" to see how each datapoint differs in score between the original model (1) and debiased model (2). Points off the diagonal have differences in results between the two models.
  - Are there patterns of which datapoints don't agree between the two models?
  - If you set the ground truth feature dropdown in the "Performance + Fairness" tab to "is_toxic", then you can color or bin the datapoints by "inference correct 1" or "inference correct 2". Are there patterns of which datapoints are incorrect for model 1? For model 2?

You may want to focus on terms listed [here](https://github.com/conversationai/unintended-ml-bias-analysis/blob/master/unintended_ml_bias/bias_madlibs_data/adjectives_people.txt)